In [13]:
import spacy
import time 
from gtts import gTTS 
import speech_recognition as sr
from pathlib import Path
from pprint import pprint
from playsound import playsound
import datetime

nlp = spacy.load('en_core_web_sm')

In [5]:
reg_menu = ['Chicken', 'Burger', 'Salmon']
vegetarian_menu = ['Pasta', 'Tacos', 'Chickpea']
all_menu = reg_menu + vegetarian_menu

words = ['Do','Have','Which','What', 'Do']
no_interest = ['nothing', 'none']
doc = []
list_of_words = []

## Beginning of Functions

In [19]:
def text_to_speech(mytext, language):
    speech = gTTS(text = mytext, lang = language)
    date_string = datetime.datetime.now().strftime("%d%m%Y%H%M%S")
    filename = "voice"+date_string+".mp3"    

    speech.save(filename)
    playsound(filename)

In [7]:
def sr_mic():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak:")
        audio = r.listen(source)
        print("You said: " + r.recognize_google(audio))
        text = r.recognize_google(audio)
        text = text.replace('-', ' ')
    return text

The welcome_menu() reads the available menu based on the customer preferencee, and further ask them which of the read-out meal(s) they would love to have. Special words like *none*, *nothing* are being detected, and the Robot gives a final message afterwards.

In [8]:
# function reads the right menu based on customer preference
def welcome_menu():
    customer_speech = sr_mic()

    if "regular" in customer_speech:
        text_to_speech('Good choice. These are the regular meals for tonight', 'en')
        [text_to_speech(meal, 'en') for meal in reg_menu]
        text_to_speech("Which would you love to have?", 'en')
        return "regular"
    elif "vegetarian" in customer_speech:
        text_to_speech('Okay, these are the vegetarian meals we have', 'en')
        [text_to_speech(meal, 'en') for meal in vegetarian_menu]
        text_to_speech("Which would you love to have?", 'en')
        return "vegetarian"
    elif any(word in customer_speech for word in no_interest):
        text_to_speech("Thanks for stopping by. We hope to see you again soon", 'en')

In [9]:
def repeat_menu():
    text_to_speech('Here is what we have for Regular meals ', 'en')
    for meal in reg_menu:
        text_to_speech(meal, 'en')

    text_to_speech('Here is what we have for Vegetarian meals ', 'en')
    for meal in vegetarian_menu:
        text_to_speech(meal, 'en')
        
    text_to_speech('What would you like?', 'en')

The take_customer_order() function takes in an argument which is the customer speech. This sentence is then processed in order to use the spacy pasrer tool to distinguish *parts of the speech* of the sentence. Since we know food belongs to either *NOUN* or *PROPN*. We take this advantage of Part-of-speech tagging to identify the meals from the Customer speech, and we also do an extra check to confirm the meal can be found in our list of Available meals.

In [10]:
# function that takes order from customer
def take_customer_order(customer_speech):
    global list_of_words
    list_of_words = []
    doc = nlp(customer_speech)
    spacy.displacy.render(doc, style='dep', jupyter=True)
    
    for possible_subject in doc:
        print("doc --", possible_subject.pos_)
        if ((possible_subject.pos_ == 'NOUN' or possible_subject.pos_ == 'PROPN') and possible_subject.text.title() in all_menu):
            list_of_words.append(possible_subject.text.title())

    if(len(list_of_words) == 0):
        for word in doc:
            if(word.text.title() in words):
                list_of_words.append(word.text.title())
                break

    if(len(list_of_words) == 0):
        for word in doc:
            list_of_words.append(word.text.title())
            break

    list_of_words = list(set(list_of_words))
    print("Menus ", list_of_words)
    return doc


The confirm_customer_order() is the final function invoked to prepare the customer order and serve them their meal. In the case where 

In [11]:
def confirm_customer_order():
    global list_of_words
    #Handling multiple orders
    orders = [] 

    for meal in list_of_words:
        # verify the preference belongs to the right menu category
        if (order_preference == 'regular' and meal in reg_menu) or (order_preference == 'vegetarian' and meal in vegetarian_menu):
            orders.append(meal)
            
        # misplaced meal
        elif (order_preference == 'regular' and meal in vegetarian_menu) or (order_preference == 'vegetarian' and meal in reg_menu):
            text_to_speech('You have made the wrong order, please try again','en')
            order = sr_mic()
            take_customer_order(order)
            confirm_customer_order()

    if len(orders) > 1:
        speechoutput = " and ".join(str(x) for x in orders)
        text_to_speech('Here are your '+ speechoutput, 'en')
        text_to_speech('Enjoy your meal!', 'en')
        print('Here are your '+ speechoutput)
        print('Enjoy your meal!')
        return True

    for meal in list_of_words:
        #If the meal the customer ordered is available
        if meal in all_menu:
            print('You want ', meal)
            text_to_speech('Here is your '+ meal, 'en')
            text_to_speech('Enjoy your meal!', 'en')
            print('Here is your '+ meal)
            print('Enjoy your meal!')
            break;

        #If customer asks about the meals again or not sure of what to have
        elif meal in words:
            repeat_menu()
            list_of_words = []
            order = sr_mic()
            take_customer_order(order)
            confirm_customer_order()
            break

        #If customer doesn't want the available meals
        elif meal in no_interest:
            list_of_words = []
            text_to_speech('Thanks for stopping by. We hope to see you again soon', 'en')
            break

        #If customer request for something that is not available in the menu list
        elif meal not in all_menu and words:
            print('I\'m sorry but we don\'t have '+meal)
            text_to_speech('I\'m sorry We don\'t have '+meal, 'en')
            repeat_menu()
            list_of_words = []
            order = sr_mic()
            take_customer_order(order)
            confirm_customer_order()
            break

# Beginning of User Interaction


In [146]:
speech = 'Hello! Welcome to Hard rock cafe. Would you like our Regular or Vegetarian menu?'
text_to_speech(speech, 'en')

### USER INTERACTION:

You have to tell the Robot your preference, i.e if you want the regular or vegetarian menu

In [24]:
order_preference = welcome_menu()
print("Order preference is", order_preference)

Speak:
You said: I want regular
Order preference is regular


### USER INTERACTION:

Now, you have to pick the options from the menu list that the Robot read out. SPACY parser draws the dependency trees for user' sentence to give a better understanding of the sentence.

In [27]:
customer_speech = sr_mic()
customer_order = take_customer_order(customer_speech)

Speak:
You said: burger and Chicken please


doc -- PROPN
doc -- CCONJ
doc -- PROPN
doc -- INTJ
Menus  ['Burger', 'Chicken']


In [28]:
print(customer_order)    
print(list_of_words)

burger and Chicken please
['Burger', 'Chicken']


The robot confirms the order, and brings the meal to the customer table.

In [29]:
confirm_customer_order()

Here are your Burger and Chicken
Enjoy your meal!


True